In [1]:
from KG_trainer_w_comp2 import get_KG_trainer

c:\Users\quent\anaconda3\envs\capstone_scratch_2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading cpnet...
Done
Loaded 3947 allowed concept IDs from train/val data.


In [2]:
DATA_PATH = "data/eg"
source_path=f"{DATA_PATH}/train.source"
target_path=f"{DATA_PATH}/train.target"
model_name = "facebook/bart-base"
output_dir = "KG_finetuned_out2"
max_len = 128
epochs = 3
train_batch_size = 10
num_points = 200

In [3]:
trainer = get_KG_trainer(
    source_path=source_path,
    target_path=target_path,
    model_name=model_name,
    output_dir=output_dir,
    max_len=max_len,
    epochs=epochs,
    train_batch_size=train_batch_size,
    num_points=num_points
)

Loading preprocessed dataset from disk...


Some weights of BartGraphAwareForConditionalGeneration were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['graph_encoder.edge_embedding.weight', 'graph_encoder.embedding.weight', 'graph_encoder.gcn.linear.bias', 'graph_encoder.gcn.linear.weight', 'graph_encoder.sag_pool.attn.bias', 'graph_encoder.sag_pool.attn.weight', 'graph_encoder.sag_pool.scoring.gcn.linear.bias', 'graph_encoder.sag_pool.scoring.gcn.linear.weight', 'graph_encoder.sag_pool.scoring.linear.bias', 'graph_encoder.sag_pool.scoring.linear.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\quent\OneDrive\Desktop\Code\Efficient-Common-Sense-in-LLMs-via-Knowledge-Graph-Compression\KG_trainer_w_comp2.py:456: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [4]:
print(trainer.train_dataset)

Dataset({
    features: ['source', 'target', 'input_ids', 'attention_mask', 'labels', 'concept_ids', 'edge_index', 'edge_type'],
    num_rows: 200
})


In [5]:
#4m 6.4s

In [6]:
trainer.train()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
trainer.save_model(output_dir + "/final_model")

In [ ]:
from KG_trainer_w_comp import BartGraphAwareForConditionalGeneration
from transformers import BartTokenizer
import torch

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model_path = "KG_finetuned_out2/checkpoint-30"
tokenizer = BartTokenizer.from_pretrained(model_path)
model = BartGraphAwareForConditionalGeneration.from_pretrained(model_path)
model.to(device)
model.eval() 

BartGraphAwareForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          

In [ ]:

from KG_trainer_w_comp import get_graph_info, BartGraphAwareForConditionalGeneration
from transformers import BartTokenizer
import torch

# Path to your fine-tuned model checkpoint
FINE_TUNED_MODEL_DIR = "KG_finetuned_out2/checkpoint-30"

# Load the tokenizer and custom model.
tokenizer = BartTokenizer.from_pretrained(FINE_TUNED_MODEL_DIR)
model = BartGraphAwareForConditionalGeneration.from_pretrained(FINE_TUNED_MODEL_DIR)

# Move model to GPU if available.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def generate_explanation_kg_compressed(sentence: str) -> list:
    # Tokenize the input sentence explicitly (ensure input_ids are produced).
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, max_length=128)
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)
    
    # Extract the full graph information from the sentence.
    graph_info = get_graph_info(sentence)
    # Ensure the graph tensors have a batch dimension.
    if graph_info["concept_ids"].dim() == 1:
        graph_info["concept_ids"] = graph_info["concept_ids"].unsqueeze(0)
    if graph_info["edge_index"].dim() == 2:
        graph_info["edge_index"] = graph_info["edge_index"].unsqueeze(0)
    if graph_info["edge_type"].dim() == 1:
        graph_info["edge_type"] = graph_info["edge_type"].unsqueeze(0)
        
    # Move the graph tensors to the correct device.
    graph_info["concept_ids"] = graph_info["concept_ids"].to(device)
    graph_info["edge_index"] = graph_info["edge_index"].to(device)
    graph_info["edge_type"] = graph_info["edge_type"].to(device)
    
    # Call generate() explicitly with input_ids and attention_mask.
    output_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        concept_ids=graph_info["concept_ids"],
        edge_index=graph_info["edge_index"],
        edge_type=graph_info["edge_type"],
        max_length=60,
        num_beams=3,
        num_return_sequences=3,
        early_stopping=True
    )
    
    # Decode the generated token IDs.
    explanations = [tokenizer.decode(ids, skip_special_tokens=True) for ids in output_ids]
    return explanations


c:\Users\quent\anaconda3\envs\capstone_scratch_2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading cpnet...
Done
Loaded 3947 allowed concept IDs from train/val data.
Explanation 1: People need to sleep.
Explanation 2: There is no need to sleep.
Explanation 3: People need to sleep to sleep.


In [ ]:
input_sentence = "We use book to know the time."
generate_explanation_kg_compressed(input_sentence)

['Book is used to know the time.',
 'Book is a book.',
 'Book is used to know time.']

In [ ]:
input_sentence = "she ate a brick."
generate_explanation_kg_compressed(input_sentence)

['Brick is not a food.', 'A brick is not a food.', 'Bricks are not edible.']

In [ ]:
#these are pretty bad but I only trained it for 30 batches of 20 datapoints = 600 datapoints so this is expected
#the other model were trained on 3 epochs of 25000 datapoints = ~75000 datapoints total